In [1]:
import instaloader
import pandas as pd
from datetime import datetime
from itertools import takewhile, dropwhile
import orjson

def get_instagram_posts(username, startdate, enddate, instaloader_username, instaloader_password):
    """
    This function logs into Instagram using Instaloader, retrieves posts from a given profile within a specified date range,
    and returns a list of posts' metadata as dictionaries.
    
    Parameters:
    - username: Instagram username of the profile to scrape.
    - startdate: The start date of the posts to scrape.
    - enddate: The end date of the posts to scrape.
    - instaloader_username: Your Instagram username for logging in.
    - instaloader_password: Your Instagram password for logging in.
    
    Returns:
    - posts_data: A list of dictionaries containing post metadata.
    """
    # Create an Instaloader object with parameters
    L = instaloader.Instaloader(download_pictures=False, download_videos=False, 
                                download_comments=False, compress_json=False)
    
    # Log in with the Instaloader object
    L.login(instaloader_username, instaloader_password)
    
    # Search the Instagram profile
    profile = instaloader.Profile.from_username(L.context, username)
    
    posts_data = []
    
    # Scrape the posts
    posts = profile.get_posts()
    for post in takewhile(lambda p: p.date > startdate, dropwhile(lambda p: p.date > enddate, posts)):
        post_info = {
            'Time': post.date,
            'Content Type': post.typename,
            'Likes': post.likes,
            'Comments': post.comments,
            'Caption': post.caption,
            'Post URL': post.url,
            'Hashtags': ', '.join(post.caption_hashtags),
            'Video Views': post.video_view_count if post.typename == 'video' else None
        }
        posts_data.append(post_info)
    
    return posts_data

def create_dataframe(posts_data):
    """
    This function creates a DataFrame from the list of posts' metadata.
    
    Parameters:
    - posts_data: A list of dictionaries containing post metadata.
    
    Returns:
    - dataframe: A Pandas DataFrame containing the posts' data.
    """
    dataframe = pd.DataFrame(posts_data)
    return dataframe

# Usage Example

# Instagram username and date range for scraping
target_username = "influencer_username"
startdate = datetime(2024, 8, 1)
enddate = datetime(2024, 8, 7)

# Instaloader credentials (replace with your actual credentials)
instaloader_username = "your_username"
instaloader_password = "your_instagram_password*"

# Get Instagram posts data
posts_data = get_instagram_posts(target_username, startdate, enddate, instaloader_username, instaloader_password)

# Create a DataFrame
df_instagram = create_dataframe(posts_data)

# Save the DataFrame to an Excel file
df_instagram.to_excel(f"{target_username}_instagram_data.xlsx", index=False)

print(f"Instagram data for {target_username} has been saved to {target_username}_instagram_data.xlsx")